# Metrics

** Plan de mejora, ponerlo en funciones

In [1]:
import pandas as pd
import numpy as np
import glob
import os

from geopy.distance import geodesic

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error

from scipy.stats import spearmanr

import warnings
warnings.filterwarnings('ignore')


In [2]:
arroz_all = pd.read_parquet('data/prec_monthly_obs_sat.parquet')
#se añade las categorias del semaforo
clas = pd.read_excel('data/semaforo.xlsx')

In [3]:
arroz_all= arroz_all.merge(clas[['dpto','station', 'clasificacion']], how='left', on=['station', 'dpto'])

# General Metrics

In [4]:
# Metricas
def kling_gupta(obs, pred):
    cc = np.corrcoef(obs, pred)[0, 1]  # Correlación de Pearson
    std_obs = np.std(obs)
    std_pred = np.std(pred)
    
    # Evitar división por cero
    if std_obs == 0 or std_pred == 0:
        alpha = np.nan  # Ratio de desviación estándar indefinido
    else:
        alpha = std_pred / std_obs  # Ratio de desviación estándar

    mean_obs = np.mean(obs)
    mean_pred = np.mean(pred)
    
    # Evitar división por cero
    if mean_obs == 0:
        beta = np.nan  # Ratio de medias indefinido
    else:
        beta = mean_pred / mean_obs  # Ratio de medias

    # Calcular KGE con validación de NaN
    if np.isnan(cc) or np.isnan(alpha) or np.isnan(beta):
        return np.nan
    else:
        kge = 1 - np.sqrt((cc - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
        return kge

# Función para calcular el R^2 ajustado
def r2_ajustado(r2, n, p=1):
    if n > p + 1:
        return 1 - ((1 - r2) * (n - 1) / (n - p - 1))
    else:
        return np.nan  # Retorna NaN si no es posible calcularlo, es decir no hay suficientes datos

    
    
def calcular_mape(obs, pred):
    obs_nonzero = obs[obs != 0]
    pred_nonzero = pred[obs != 0]
    return np.mean(np.abs((obs_nonzero - pred_nonzero) / obs_nonzero)) * 100

def calcular_maape(obs, pred):
    obs_nonzero = obs[obs != 0]
    pred_nonzero = pred[obs != 0]
    return np.mean(np.arctan(np.abs((obs_nonzero - pred_nonzero) / obs_nonzero)))

In [5]:
# generar las agregaciones anuales
if pd.api.types.is_period_dtype(arroz_all['month_year']):
    arroz_all['month_year'] = arroz_all['month_year'].apply(lambda x: x.to_timestamp())


arroz_all['year'] = arroz_all['month_year'].dt.year
prec_anual = arroz_all.groupby(['clasificacion','station', 'year', 'fuente'], as_index=False).agg({'prec_month': 'sum'}).rename(columns={'prec_month': 'prec_anual'})

prec_anual.head(4)


,clasificacion,station,year,fuente,prec_anual
0,A,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2011,agera5-precipitation,392.570000
1,A,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2011,chirps-precipitation,678.976980
2,A,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2011,fedearroz,759.800000
3,A,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2012,agera5-precipitation,1398.189996


In [6]:
def calcular_metricas(obs, pred):
    if len(obs) < 2 or len(pred) < 2:
        return {
            'r2': None,
            'r2_ajustado': None,
            'rmse': None,
            'mae': None,
            'std_error': None,
            'kge': None,
            'spearman': None,
            'bias': None,
            'mape': None,
            'maape': None
        }
    else:
        r2 = r2_score(obs, pred)
        n = len(obs)
        rmse = np.sqrt(mean_squared_error(obs, pred))
        mae = mean_absolute_error(obs, pred)
        
        # Calcular la diferencia y std_error considerando NaN
        diff = np.array(obs) - np.array(pred)
        if np.all(np.isnan(diff)):
            std_error = np.nan
        else:
            std_error = np.nanstd(diff)

        # Confirmar std_error = 0 si los errores son constantes
        if std_error == 0:
            std_error = 0
        
        return {
            'r2': r2,
            'r2_ajustado': r2_ajustado(r2, n),
            'rmse': rmse,
            'mae': mae,
            'std_error': std_error,
            'kge': kling_gupta(obs, pred),
            'spearman': spearmanr(obs, pred).correlation if not np.all(np.isnan(obs) | np.isnan(pred)) else np.nan,
            'bias': np.mean(pred) - np.mean(obs),
            'mape': calcular_mape(obs, pred),
            'maape': calcular_maape(obs, pred)
        }


In [7]:
metricas_estacion = []

# Iterar sobre cada estación
for estacion in prec_anual['station'].unique():
    df_filtrado = prec_anual[prec_anual['station'] == estacion]
    df_pivot = df_filtrado.pivot(index='year', columns='fuente', values='prec_anual').dropna()
    
    if all(fuente in df_pivot.columns for fuente in ['fedearroz', 'agera5-precipitation', 'chirps-precipitation']):
        metricas_agera = calcular_metricas(df_pivot['fedearroz'], df_pivot['agera5-precipitation'])
        metricas_chirps = calcular_metricas(df_pivot['fedearroz'], df_pivot['chirps-precipitation'])
        
        metricas_estacion.append({
            'clasificacion': arroz_all[(arroz_all['station'] == estacion)]['clasificacion'].unique()[0],
            'departamento': arroz_all[(arroz_all['station'] == estacion)]['dpto'].unique()[0],
            'municipio': arroz_all[(arroz_all['station'] == estacion)]['mun'].unique()[0],
            'station': estacion,
            'r2_agera': metricas_agera['r2'],
            'r2_ajustado_agera': metricas_agera['r2_ajustado'],
            'rmse_agera': metricas_agera['rmse'],
            'mae_agera': metricas_agera['mae'],
            'std_error_agera': metricas_agera['std_error'],  
            'kge_agera': metricas_agera['kge'],
            'spearman_agera': metricas_agera['spearman'],
            'bias_agera': metricas_agera['bias'],
            'mape_agera': metricas_agera['mape'],
            'maape_agera': metricas_agera['maape'],
            'r2_chirps': metricas_chirps['r2'],
            'r2_ajustado_chirps': metricas_chirps['r2_ajustado'],
            'rmse_chirps': metricas_chirps['rmse'],
            'mae_chirps': metricas_chirps['mae'],
            'std_error_chirps': metricas_chirps['std_error'],  # sol fede
            'kge_chirps': metricas_chirps['kge'],
            'spearman_chirps': metricas_chirps['spearman'],
            'bias_chirps': metricas_chirps['bias'],
            'mape_chirps': metricas_chirps['mape'],
            'maape_chirps': metricas_chirps['maape']
        })


df_metricas_estacion = pd.DataFrame(metricas_estacion)
df_metricas_estacion.head(3)

,clasificacion,departamento,municipio,station,r2_agera,r2_ajustado_agera,rmse_agera,mae_agera,std_error_agera,kge_agera,...,r2_chirps,r2_ajustado_chirps,rmse_chirps,mae_chirps,std_error_chirps,kge_chirps,spearman_chirps,bias_chirps,mape_chirps,maape_chirps
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,0.001535,-0.089235,648.273641,476.130769,629.298960,0.138778,...,0.065542,-0.019409,627.150655,455.459175,607.499815,0.362725,0.582418,155.762379,45.807979,0.387382
1,A,TOLIMA,IBAGUE,FEDEARROZ_IBAGUE_EL_CHACO,-0.193490,-0.301990,590.522932,500.239231,548.077913,0.035072,...,0.297486,0.233621,453.059118,381.923986,437.289058,0.443740,0.510989,118.494069,47.375607,0.387477
2,A,TOLIMA,IBAGUE,FEDEARROZ_IBAGUE_PERALES,-0.125517,-0.227836,458.104123,352.705384,402.884099,0.230820,...,0.400298,0.345780,334.391954,278.242200,250.481497,0.735890,0.763736,221.533290,32.919387,0.297633


### mejorar el formato de salida

In [8]:
def format_dataframe(df):
    #  agera
    df_agera = df[['clasificacion','departamento', 'municipio', 'station',  'r2_agera', 'r2_ajustado_agera', 'rmse_agera', 'mae_agera', 
                   'std_error_agera', 'kge_agera', 'spearman_agera', 'bias_agera','mape_agera','maape_agera']].copy()
    df_agera['source'] = 'agera5'
    df_agera.columns = ['clasificacion','departamento', 'municipio','station', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std', 'kge', 'spearman', 'bias','mape','maape', 'source']
    
    #  chirps
    df_chirps = df[['clasificacion','departamento', 'municipio','station',  'r2_chirps', 'r2_ajustado_chirps', 'rmse_chirps', 'mae_chirps', 
                    'std_error_chirps', 'kge_chirps', 'spearman_chirps', 'bias_chirps','mape_chirps','maape_chirps']].copy()
    df_chirps['source'] = 'chirps'
    df_chirps.columns = ['clasificacion','departamento', 'municipio','station',  'r2', 'r2_ajustado', 'rmse', 'mae', 'std', 'kge', 'spearman', 'bias', 'mape','maape','source']

    df_format = pd.concat([df_agera, df_chirps], ignore_index=True)
    
    return df_format

df_metricas = format_dataframe(df_metricas_estacion)
df_metricas.head(3)

,clasificacion,departamento,municipio,station,r2,r2_ajustado,rmse,mae,std,kge,spearman,bias,mape,maape,source
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,0.001535,-0.089235,648.273641,476.130769,629.298960,0.138778,0.390110,-155.696924,45.456140,0.387893,agera5
1,A,TOLIMA,IBAGUE,FEDEARROZ_IBAGUE_EL_CHACO,-0.193490,-0.301990,590.522932,500.239231,548.077913,0.035072,0.126374,-219.836154,57.527867,0.402469,agera5
2,A,TOLIMA,IBAGUE,FEDEARROZ_IBAGUE_PERALES,-0.125517,-0.227836,458.104123,352.705384,402.884099,0.230820,0.302198,-218.045385,34.286018,0.303928,agera5


In [9]:
df_metricas.isna().sum()

clasificacion    0
departamento     0
municipio        0
station          0
r2               0
r2_ajustado      0
rmse             0
mae              0
std              0
kge              0
spearman         0
bias             0
mape             0
maape            0
source           0
dtype: int64

In [10]:
df_metricas.to_parquet('data/overall_metrics.parquet')

# Monthly metrics

In [11]:
metricas_mensuales = []


for estacion in arroz_all['station'].unique():
    for mes in range(1, 13): 
        df_filtrado = arroz_all[(arroz_all['station'] == estacion) & (arroz_all['month_year'].dt.month == mes)]

        
        df_pivot = df_filtrado.pivot(index=['station', 'month_year'], 
                                     columns='fuente', 
                                     values='prec_month').dropna()

        
        if all(fuente in df_pivot.columns for fuente in ['fedearroz', 'agera5-precipitation', 'chirps-precipitation']):
            metricas_agera = calcular_metricas(df_pivot['fedearroz'], df_pivot['agera5-precipitation'])
            metricas_chirps = calcular_metricas(df_pivot['fedearroz'], df_pivot['chirps-precipitation'])
            
            
            metricas_mensuales.append({
                'clasificacion': arroz_all[(arroz_all['station'] == estacion)]['clasificacion'].unique()[0],
                'departamento': arroz_all[(arroz_all['station'] == estacion)]['dpto'].unique()[0],
                'municipio': arroz_all[(arroz_all['station'] == estacion)]['mun'].unique()[0],
                'station': estacion,
                'mes': mes,
                'r2_agera': metricas_agera['r2'],
                'r2_ajustado_agera': metricas_agera['r2_ajustado'],
                'rmse_agera': metricas_agera['rmse'],
                'mae_agera': metricas_agera['mae'],
                'std_error_agera': metricas_agera['std_error'],
                'kge_agera': metricas_agera['kge'],
                'spearman_agera': metricas_agera['spearman'],
                'bias_agera': metricas_agera['bias'],
                'mape_agera': metricas_agera['mape'],
                'maape_agera': metricas_agera['maape'],
                'r2_chirps': metricas_chirps['r2'],
                'r2_ajustado_chirps': metricas_chirps['r2_ajustado'],
                'rmse_chirps': metricas_chirps['rmse'],
                'mae_chirps': metricas_chirps['mae'],
                'std_error_chirps': metricas_chirps['std_error'],
                'kge_chirps': metricas_chirps['kge'],
                'spearman_chirps': metricas_chirps['spearman'],
                'bias_chirps': metricas_chirps['bias'],
                'mape_chirps': metricas_chirps['mape'],
                'maape_chirps': metricas_chirps['maape']
            })


df_metricas_mensuales = pd.DataFrame(metricas_mensuales)
df_metricas_mensuales.head(2)

,clasificacion,departamento,municipio,station,mes,r2_agera,r2_ajustado_agera,rmse_agera,mae_agera,std_error_agera,...,r2_chirps,r2_ajustado_chirps,rmse_chirps,mae_chirps,std_error_chirps,kge_chirps,spearman_chirps,bias_chirps,mape_chirps,maape_chirps
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,1,-0.432291,-0.611327,104.202734,85.149,97.009215,...,0.585485,0.533670,56.057519,46.175012,56.001611,0.647108,0.357576,-2.503003,35.470575,0.316023
1,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2,-0.219753,-0.372223,67.194785,59.931,61.914935,...,0.528625,0.469703,41.771757,37.004905,37.011374,0.737772,0.745455,19.365895,43.839146,0.350530


In [12]:
df_metricas_mensuales.r2_agera.isna().sum() # estos son los datos de la estación que solo tenia una observacion en su historia en esos meses

6

# ISSUE: Esto es porque las metrics necesitan más de un dato para ser calculadas

In [13]:
df_metricas_mensuales[df_metricas_mensuales.r2_agera.isna()]

,clasificacion,departamento,municipio,station,mes,r2_agera,r2_ajustado_agera,rmse_agera,mae_agera,std_error_agera,...,r2_chirps,r2_ajustado_chirps,rmse_chirps,mae_chirps,std_error_chirps,kge_chirps,spearman_chirps,bias_chirps,mape_chirps,maape_chirps
421,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,E,NORTE DE SANTANDER,CUCUTA,FEDEARROZ_CUCUTA_LA_JAVILLA,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
## dejar el mismo formato que las generales
def format_dataframe(df):
    #  agera
    df_agera = df[['clasificacion','departamento', 'municipio', 'station', 'mes', 'r2_agera', 'r2_ajustado_agera', 'rmse_agera', 'mae_agera', 
                   'std_error_agera', 'kge_agera', 'spearman_agera', 'bias_agera','mape_agera','maape_agera']].copy()
    df_agera['source'] = 'agera5'
    df_agera.columns = ['clasificacion','departamento', 'municipio','station', 'mes', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_error', 'kge', 'spearman', 'bias','mape','maape', 'source']
    
    #  chirps
    df_chirps = df[['clasificacion','departamento', 'municipio','station', 'mes', 'r2_chirps', 'r2_ajustado_chirps', 'rmse_chirps', 'mae_chirps', 
                    'std_error_chirps', 'kge_chirps', 'spearman_chirps', 'bias_chirps','mape_chirps','maape_chirps']].copy()
    df_chirps['source'] = 'chirps'
    df_chirps.columns = ['clasificacion','departamento', 'municipio','station', 'mes', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_error', 'kge', 'spearman', 'bias', 'mape','maape','source']
    
    # Concatenar ambos dataframes
    df_format = pd.concat([df_agera, df_chirps], ignore_index=True)
    
    return df_format


df_metricas_mensuales= format_dataframe(df_metricas_mensuales)
df_metricas_mensuales.head()

,clasificacion,departamento,municipio,station,mes,r2,r2_ajustado,rmse,mae,std_error,kge,spearman,bias,mape,maape,source
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,1,-0.432291,-0.611327,104.202734,85.149000,97.009215,0.082316,-0.066667,-38.045000,50.349702,0.446402,agera5
1,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2,-0.219753,-0.372223,67.194785,59.931000,61.914935,0.222341,0.163636,-26.109000,69.225511,0.504275,agera5
2,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,3,0.024200,-0.084222,113.514574,96.960000,113.409082,0.267899,0.527273,4.892728,76.020062,0.540922,agera5
3,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,4,-0.203629,-0.337365,74.644062,48.032727,65.958708,0.428948,0.518182,34.945454,229.735901,0.420815,agera5
4,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,5,-0.091494,-0.200644,584.099227,249.586666,576.888700,-0.616465,0.020979,-91.495000,268.901403,0.916252,agera5


In [15]:
df_metricas_mensuales.to_parquet('data/monthly_metrics.parquet')

In [16]:
df_metricas_mensuales.station.nunique()

42

## transform metrics a un formato tipo reporte

In [18]:
# Crear el pivote
df_pivot = df_metricas_mensuales.pivot_table(
    index=['clasificacion', 'departamento', 'municipio', 'station'],  # Índices
    columns=['mes', 'source'],  # Niveles de columnas
    values=['r2_ajustado', 'rmse', 'kge', 'spearman', 'bias', 'mape', 'maape'],  # Métricas
    aggfunc='first'
)

# Reorganizar niveles para que el nivel de métricas sea el nivel más profundo (nivel 3)
df_pivot.columns = df_pivot.columns.reorder_levels([0, 1, 2])


# Ordenar los niveles de columnas para mostrar los meses en orden adecuado
df_pivot = df_pivot.sort_index(axis=1, level=0)

# Mostrar el resultado
df_pivot.head()


bias  \
mes                                                                             1    
source                                                                      agera5   
clasificacion departamento municipio   station                                       
A             HUILA        CAMPOALEGRE FEDEARROZ_CAMPOALEGRE_ALTAGRACIA -38.045000   
              TOLIMA       IBAGUE      FEDEARROZ_IBAGUE_EL_CHACO        -16.242500   
                                       FEDEARROZ_IBAGUE_PERALES         -27.714445   
                           LERIDA      FEDEARROZ_LERIDA_LA_CARMELITA    -39.563333   
                           PRADO       FEDEARROZ_PRADO_ASOPRADO          16.758749   

                                                                                    \
mes                                                                                  
source                                                                      chirps   
clasificacion departamento municipio   station                                       
A             HUILA        CAMPOALEGRE FEDEARROZ_CAMPOALEGRE_ALTAGRACIA  -2.503003   
              TOLIMA       IBAGUE      FEDEARROZ_IBAGUE_EL_CHACO         21.286746   
                                       FEDEARROZ_IBAGUE_PERALES          17.953991   
                           LERIDA      FEDEARROZ_LERIDA_LA_CARMELITA     45.099786   
                           PRADO       FEDEARROZ_PRADO_ASOPRADO          15.400021   

                                                                                    \
mes                                                                             2    
source                                                                      agera5   
clasificacion departamento municipio   station                                       
A             HUILA        CAMPOALEGRE FEDEARROZ_CAMPOALEGRE_ALTAGRACIA -26.109000   
              TOLIMA       IBAGUE      FEDEARROZ_IBAGUE_EL_CHACO        -16.659091   
                                       FEDEARROZ_IBAGUE_PERALES         -23.396000   
                           LERIDA      FEDEARROZ_LERIDA_LA_CARMELITA    -87.402500   
                           PRADO       FEDEARROZ_PRADO_ASOPRADO          -5.454999   

                                                                                    \
mes                                                                                  
source                                                                      chirps   
clasificacion departamento municipio   station                                       
A             HUILA        CAMPOALEGRE FEDEARROZ_CAMPOALEGRE_ALTAGRACIA  19.365895   
              TOLIMA       IBAGUE      FEDEARROZ_IBAGUE_EL_CHACO         16.744264   
                                       FEDEARROZ_IBAGUE_PERALES          30.256793   
                           LERIDA      FEDEARROZ_LERIDA_LA_CARMELITA     33.437283   
                           PRADO       FEDEARROZ_PRADO_ASOPRADO          17.812058   

                                                                                     \
mes                                                                              3    
source                                                                       agera5   
clasificacion departamento municipio   station                                        
A             HUILA        CAMPOALEGRE FEDEARROZ_CAMPOALEGRE_ALTAGRACIA    4.892728   
              TOLIMA       IBAGUE      FEDEARROZ_IBAGUE_EL_CHACO         -28.495834   
                                       FEDEARROZ_IBAGUE_PERALES          -43.826000   
                           LERIDA      FEDEARROZ_LERIDA_LA_CARMELITA    -115.321429   
                           PRADO       FEDEARROZ_PRADO_ASOPRADO           58.462501   

                                                                                     \
mes                                                                                   
source              

In [19]:
df_pivot.to_excel('data/metricas_pivot_mensual.xlsx')

## método de selección de métricas--- falta recibir feedback del equipo fedearroz!, antes mostrar las graficas

| Métrica        | Rango                | Mejor valor   | Interpretación                         |
|----------------|----------------------|---------------|-----------------------------------------------|
| **Bias**       | (-∞, ∞)              | 0             | Positivo = sobreestimación, negativo = subestimación |
| **KGE**        | (-∞, 1]              | 1             | 1 es perfecto, negativo indica mal desempeño  |
| **MAAPE**      | [0, 1.57]            | 0             | Más bajo es mejor, cercano a 1.57 indica peores errores |
| **MAPE**       | [0, ∞)               | 0             | Porcentajes más bajos son mejores             |
| **R² ajustado**| (-∞, 1]              | 1             | Explica la variabilidad de los datos, negativo indica mal desempeño |
| **RMSE**       | [0, ∞)               | 0             | Cuanto más bajo, mejor                        |
| **Spearman**   | [-1, 1]              | 1             | Relación monótona perfecta es 1               |


In [20]:
## por ideas de fedearroz dejar las metricas mas simples, que no entre en redundancia y asi sesgo

def calcular_score(df):
    # Inicializar listas para almacenar los resultados
    resultados = []

    # Iterar sobre cada grupo de estación y mes
    for (clasificacion, departamento, municipio, station, mes), grupo in df.groupby(['clasificacion', 'departamento', 'municipio', 'station', 'mes']):
        
        
        chirps = grupo[grupo['source'] == 'chirps']
        agera = grupo[grupo['source'] == 'agera5']
        #print(f'este es chirps {chirps}')
        #print(agera)
        # checkpoint, deben estar los dos
        if not chirps.empty and not agera.empty:
            # Calcular puntos para agera en cada métrica
            puntos_agera = sum([
                agera['maape'].values[0] < chirps['maape'].values[0],
                agera['spearman'].values[0] > chirps['spearman'].values[0],
                agera['rmse'].values[0] < chirps['rmse'].values[0],
                agera['r2_ajustado'].values[0] > chirps['r2_ajustado'].values[0]
            ])
            #print(puntos_agera)
            
            
            if puntos_agera > 2:
                source_ganadora = 'agera5'
                metricas_ganadoras = agera[['maape', 'spearman', 'rmse', 'r2_ajustado']].iloc[0].to_dict()
            else:
                source_ganadora = 'chirps'
                metricas_ganadoras = chirps[['maape', 'spearman', 'rmse', 'r2_ajustado']].iloc[0].to_dict()

            # Crear el diccionario de resultados
            resultado = {
                'clasificacion': clasificacion,
                'departamento': departamento,
                'municipio': municipio,
                'station': station,
                'mes': mes,
                'source_seleccionada': source_ganadora,
                **metricas_ganadoras  # Desempaqueta los valores de las métricas ganadoras
            }

            # Añadir el resultado al listado
            resultados.append(resultado)

    # Crear el DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

# Llamar a la función con tu DataFrame
df_metricas_mejores = calcular_score(df_metricas_mensuales)


In [21]:
df_metricas_mejores

,clasificacion,departamento,municipio,station,mes,source_seleccionada,maape,spearman,rmse,r2_ajustado
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,1,chirps,0.316023,0.357576,56.057519,0.533670
1,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2,chirps,0.350530,0.745455,41.771757,0.469703
2,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,3,chirps,0.399603,0.736364,73.626968,0.543870
3,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,4,chirps,0.533825,0.900000,62.491260,0.062658
4,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,5,chirps,0.893581,0.384615,569.698511,-0.142171
...,...,...,...,...,...,...,...,...,...,...
499,F,CESAR,VALLEDUPAR,FEDEARROZ_VALLEDUPAR_LA_ESPERANZA,8,chirps,0.811883,-0.428571,85.339423,-1.957324
500,F,CESAR,VALLEDUPAR,FEDEARROZ_VALLEDUPAR_LA_ESPERANZA,9,chirps,0.778077,-0.238095,86.896597,-1.392576
501,F,CESAR,VALLEDUPAR,FEDEARROZ_VALLEDUPAR_LA_ESPERANZA,10,chirps,0.524395,0.372727,108.356828,-1.517232
502,F,CESAR,VALLEDUPAR,FEDEARROZ_VALLEDUPAR_LA_ESPERANZA,11,chirps,0.662930,-0.054545,91.715562,-2.972906


In [22]:
df_metricas_mejores.source_seleccionada.value_counts()

source_seleccionada
chirps    370
agera5    134
Name: count, dtype: int64

In [23]:
df_metricas_mejores.station.nunique()

42

In [24]:
df_metricas_mejores.to_parquet('df_metricas_score.parquet')

# quarterly metrics

In [25]:
def obtener_trimestre(mes):
    if mes in [1, 2, 3]:
        return 'EFM'
    elif mes in [4, 5, 6]:
        return 'AMJ'
    elif mes in [7, 8, 9]:
        return 'JAS'
    elif mes in [10, 11, 12]:
        return 'OND'


metricas_3m = []


for estacion in arroz_all['station'].unique():
    for trimestre, meses in {'EFM': [1, 2, 3],
                             'AMJ': [4, 5, 6],
                             'JAS': [7, 8, 9],
                             'OND': [10, 11, 12]}.items():
        
        df_filtrado = arroz_all[(arroz_all['station'] == estacion) & (arroz_all['month_year'].dt.month.isin(meses))]


        df_pivot = df_filtrado.pivot(index=['station', 'month_year'], 
                                     columns='fuente', 
                                     values='prec_month').dropna()

        
        if all(fuente in df_pivot.columns for fuente in ['fedearroz', 'agera5-precipitation', 'chirps-precipitation']):
            metricas_agera = calcular_metricas(df_pivot['fedearroz'], df_pivot['agera5-precipitation'])
            metricas_chirps = calcular_metricas(df_pivot['fedearroz'], df_pivot['chirps-precipitation'])
            
            
            metricas_3m.append({
                'categoria':arroz_all[(arroz_all['station'] == estacion)]['clasificacion'].unique()[0],
                'departamento':arroz_all[(arroz_all['station'] == estacion)]['dpto'].unique()[0],
                'municipio': arroz_all[(arroz_all['station'] == estacion)]['mun'].unique()[0],
                'station': estacion,
                'trimestre': trimestre,
                'r2_agera': metricas_agera['r2'],
                'r2_ajustado_agera': metricas_agera['r2_ajustado'],
                'rmse_agera': metricas_agera['rmse'],
                'mae_agera': metricas_agera['mae'],
                'std_error_agera': metricas_agera['std_error'],
                'kge_agera': metricas_agera['kge'],
                'spearman_agera': metricas_agera['spearman'],
                'bias_agera': metricas_agera['bias'],
                'r2_chirps': metricas_chirps['r2'],
                'r2_ajustado_chirps': metricas_chirps['r2_ajustado'],
                'rmse_chirps': metricas_chirps['rmse'],
                'mae_chirps': metricas_chirps['mae'],
                'std_error_chirps': metricas_chirps['std_error'],
                'kge_chirps': metricas_chirps['kge'],
                'spearman_chirps': metricas_chirps['spearman'],
                'bias_chirps': metricas_chirps['bias']
            })


df_metricas_3m = pd.DataFrame(metricas_3m)

In [26]:
def format_dataframe_3m(df):
    df_agera = df[['categoria','departamento', 'municipio','station', 'trimestre', 'r2_agera', 'r2_ajustado_agera', 'rmse_agera', 'mae_agera', 
                   'std_error_agera', 'kge_agera', 'spearman_agera', 'bias_agera']].copy()
    df_agera['source'] = 'agera5'
    df_agera.columns = ['categoria', 'departamento', 'municipio','station', 'trimestre', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std', 'kge', 'spearman', 'bias', 'source']
    
    df_chirps = df[['categoria', 'departamento', 'municipio','station', 'trimestre', 'r2_chirps', 'r2_ajustado_chirps', 'rmse_chirps', 'mae_chirps', 
                    'std_error_chirps', 'kge_chirps', 'spearman_chirps', 'bias_chirps']].copy()
    df_chirps['source'] = 'chirps'
    df_chirps.columns = ['categoria', 'departamento', 'municipio','station', 'trimestre', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std', 'kge', 'spearman', 'bias', 'source']
    
    df_format = pd.concat([df_agera, df_chirps], ignore_index=True)
    
    return df_format


df_metricas_3m = format_dataframe_3m(df_metricas_3m)
df_metricas_3m.head()

,categoria,departamento,municipio,station,trimestre,r2,r2_ajustado,rmse,mae,std,kge,spearman,bias,source
0,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,EFM,0.022145,-0.011574,97.629093,81.205161,95.770596,0.414461,0.374836,-18.958710,agera5
1,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,AMJ,-0.049119,-0.080911,348.268725,123.016571,348.265532,-0.313498,0.376751,1.491428,agera5
2,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,JAS,0.011922,-0.021014,49.169865,31.197812,49.168747,0.057963,0.384531,0.331563,agera5
3,A,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,OND,0.014688,-0.016103,111.086381,90.622647,101.960887,0.259574,0.452406,-44.092647,agera5
4,B,HUILA,PALERMO,FEDEARROZ_PALERMO_ASOJUNCAL,EFM,-0.170571,-0.217394,135.461225,90.033334,135.233902,0.141719,0.252137,-7.844444,agera5


In [27]:
df_metricas_3m.to_parquet('data/metricas_3m.parquet')